In [11]:
import pickle
import pandas as pd
import numpy as np
xgb_model_min_loaded = pickle.load(open('ny_min_xgb_model_min.pickle', "rb"))
xgb_model_max_loaded = pickle.load(open('ny_max_xgb_model_min.pickle', "rb"))


sel_features = ['rating', 'AI', 'AWS', 'Azure', 'Big-Data', 'C/C++', 'Data-Analysis',
       'Data-Warehouse', 'Hadoop', 'Hive', 'Java', 'Kafka', 'Linux', 'MATLAB',
       'Machine-Learning', 'Microsoft-Office', 'Microsoft-SQL-Server',
       'Natural-Language-Processing', 'NoSQL', 'Oracle', 'Pig', 'Python', 'R',
       'SAS', 'SQL', 'Scala', 'Scripting', 'Spark', 'Tableau', 'TensorFlow']

def NYC_salary_with_skills(rating,inputs_list):
    sample_list = [0] * (len(sel_features))
        input_X = pd.DataFrame([sample_list],
                               columns=list(sel_features))

        for inputs in inputs_list:
            if inputs in list(input_X.columns):
                input_X[inputs] = 1
        input_X['rating'] = float(rating)
        salary = (int(xgb_model_min_loaded.predict(input_X[:1])), int(xgb_model_max_loaded.predict(input_X[:1])))
        return salary

    skill_money = {}
    skill_money_list = []
    for skill in sel_features[1:]:
        skill_money['skill'] = skill
        skill_money['salary'] = nyc_salary_with_skills(3.6, [skill])
        skill_money_list.append(skill_money)
        skill_money = {}
    single_skill = pd.DataFrame(skill_money_list).sort_values('salary', ascending=False)

In [22]:
def NYC_salary_with_skills_and(rating, inputs_list):
    
    sample_list = [0] * (len(sel_features))
    input_x = pd.DataFrame([sample_list],
                 columns =list(sel_features))

    for inputs in inputs_list:
        if inputs in list(input_x.columns):
            input_x[inputs] = 1
    salary_min = int(xgb_model_min_loaded.predict(input_x[:1]))
    salary_max = int(xgb_model_max_loaded.predict(input_x[:1]))

    
    # suggest skill with more salary    
    suggest_list = []
    all_list = inputs_list

    for skill in single_skill['skill'][:10]:
        if skill not in inputs_list:
            suggest_list.append(skill)

        suggest_list_salary = {}
        suggest_list_salary_list = []
        for skill in suggest_list:
            all_list = []
            all_list = inputs_list+ [skill]
            suggest_list_salary['skill'] = skill
            suggest_list_salary['salary'] = int(np.subtract(NYC_salary_with_skills(rating,all_list), NYC_salary_with_skills(rating,inputs_list)).mean())
            suggest_list_salary_list.append(suggest_list_salary)
            suggest_list_salary = {}
    suggest_skills = pd.DataFrame(suggest_list_salary_list).sort_values('salary',ascending=False)
    suggest_skills = suggest_skills[suggest_skills['salary'] > 0 ]
    suggest_skills.columns = ['Skill', 'Salary increase by $']
    return {'Min_Salary': salary_min, 'Max_Salary': salary_max, 'Suggest_Skills': suggest_skills}

In [23]:
NYC_salary_with_skills_and(3.6,['MySql','R','Tableau','Python'])

{'Min_Salary': 32175,
 'Max_Salary': 57647,
 'Suggest_Skills':                   Skill  Salary increase by $
 0            TensorFlow                105208
 1                Oracle                 29744
 2      Machine-Learning                 27007
 6                   SQL                 15390
 3  Microsoft-SQL-Server                 14608
 5                   Pig                  7981
 4              Big-Data                  6840
 7                    AI                  2972}

In [26]:
rating=3.6
inputs_list=['MySql','R','Tableau','Python']

In [36]:
Suggest_Skills = NYC_salary_with_skills_and(rating, inputs_list)['Suggest_Skills']
Suggest_Skills.to_json(orient='records')

'[{"Skill":"TensorFlow","Salary increase by $":105208},{"Skill":"Oracle","Salary increase by $":29744},{"Skill":"Machine-Learning","Salary increase by $":27007},{"Skill":"SQL","Salary increase by $":15390},{"Skill":"Microsoft-SQL-Server","Salary increase by $":14608},{"Skill":"Pig","Salary increase by $":7981},{"Skill":"Big-Data","Salary increase by $":6840},{"Skill":"AI","Salary increase by $":2972}]'

In [ ]:
json.dumps